In [1]:
import xarray as xr
from pyproj import Proj
from dask.diagnostics import ProgressBar
import numpy as np
import matplotlib.pyplot as plt
import xesmf as xe
import rioxarray

In [2]:
version = '0.3'

In [3]:
def open_elevation_data():
    data = rioxarray.open_rasterio('/data/AIDL-UPC/wc2.1_30s_elev.tif')
    data = data.rename({'x': 'lon','y': 'lat'})
    data = data.assign_coords(lat=-data.lat)
    return data

# compute mean and std of the ERA5 data

In [4]:
vars_name  = ['pr', 'rh', 'tas', 'tasmax', 'tasmin', 'sfcWind'] # 'sp',, 'uas', 'vas'
path = '/home/vestella/data/NAS-data/terolink/archive/reanalysis/unversioned/era5/'

In [ ]:
lat_max = 720
for ivar in range(len(vars_name)):
    data_era5 = xr.open_mfdataset(path + vars_name[ivar]+'_day/'+vars_name[ivar] + '_*', engine = 'zarr' )
    data_era5 = data_era5.sel(time = slice(*['2000', '2018']))
    if lat_max!= data_era5.lat.shape[0]:
        data_era5 = data_era5.isel(lat = np.arange(720))
    #mean_year = data_era5[vars_name[ivar]].groupby('time.dayofyear').mean('time').load()
    data_era5 = data_era5.groupby('time.dayofyear')- data_era5.groupby('time.dayofyear').mean('time')
    if ivar ==0:
        mean = np.empty((len(vars_name), lat_max, data_era5.lon.shape[0]), dtype = np.float32)
        std = np.empty((len(vars_name), lat_max, data_era5.lon.shape[0]), dtype = np.float32)
        #seas_cyc = np.empty((len(vars_name), 366,lat_max, data_era5.lon.shape[0]), dtype = np.float32)
        mmean = np.empty((len(vars_name)), dtype = np.float64)
        sstd = np.empty((len(vars_name)), dtype = np.float64)
    
    print(data_era5.lat.values[0], data_era5.lat.values[-1])
    mean_single = data_era5[vars_name[ivar]].mean(dim = 'time').values
    std_single = data_era5[vars_name[ivar]].std(dim = 'time').values
    mean_all = data_era5[vars_name[ivar]].mean()
    std_all = data_era5[vars_name[ivar]].std()
    
    mean[ivar, :,:] = mean_single
    std[ivar, :,:] = std_single
    mmean[ivar] = mean_all
    sstd[ivar] = std_all
    #seas_cyc[ivar,:,:,:] = mean_year.values
    print(vars_name[ivar])

# store the data
da = xr.Dataset(
data_vars=dict(
    mean=([ "variable", "lat", "lon" ], mean),
    std=([ "variable", "lat", "lon" ], std),
    mean_all = ([ "variable"], mmean),
    std_all=([ "variable" ], sstd),
    #seas_cyc = ([ "variable","dayofyear", "lat", "lon" ], seas_cyc),
            ),
coords=dict(
    lon=(["lon"], data_era5.lon.values),
    lat=(["lat"], data_era5.lat.values),
    variable=vars_name,
    #dayofyear = np.arange(1,367)
),
attrs=dict(
    description="seasonal cycle, std and mean of the period 2000-2018 for each of the variables",
),
)

da.to_netcdf('/data/AIDL-UPC/std_mean_variables_ERA5_v'+version+'.nc')
print('/data/AIDL-UPC/std_mean_variables_ERA5_v'+version+'.nc')

In [5]:
lat_max = 720
for ivar in range(len(vars_name)):
    data_era5 = xr.open_mfdataset(path + vars_name[ivar]+'_day/'+vars_name[ivar] + '_*', engine = 'zarr' )
    data_era5 = data_era5.sel(time = slice(*['2000', '2018']))
    if lat_max!= data_era5.lat.shape[0]:
        data_era5 = data_era5.isel(lat = np.arange(720))
    mean_year = data_era5[vars_name[ivar]].groupby('time.dayofyear').mean('time').load()
    data_era5 = data_era5.groupby('time.dayofyear')- data_era5.groupby('time.dayofyear').mean('time')
    if ivar ==0:
        seas_cyc = np.empty((len(vars_name), 366,lat_max, data_era5.lon.shape[0]), dtype = np.float32)
    
    print(data_era5.lat.values[0], data_era5.lat.values[-1])
    seas_cyc[ivar,:,:,:] = mean_year.values
    print(vars_name[ivar])
da = xr.Dataset(
data_vars=dict(
    seas_cyc = ([ "variable","dayofyear", "lat", "lon" ], seas_cyc),
            ),
coords=dict(
    lon=(["lon"], data_era5.lon.values),
    lat=(["lat"], data_era5.lat.values),
    variable=vars_name,
    dayofyear = np.arange(1,367)
),
attrs=dict(
    description="seasonal cycle, std and mean of the period 2000-2018 for each of the variables",
),
)

da.to_netcdf('/data/AIDL-UPC/seas_cyc_variables_ERA5_v'+version+'.nc')
print('/data/AIDL-UPC/seas_cyc_variables_ERA5_v'+version+'.nc')

/home/vestella/miniconda3/envs/my_env2/lib/python3.7/site-packages/xarray/core/indexing.py:1227: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  return self.array[key]


-90.0 89.75
pr
-90.0 89.75
rh


/home/vestella/miniconda3/envs/my_env2/lib/python3.7/site-packages/xarray/core/indexing.py:1227: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  return self.array[key]


-90.0 89.75
tas


/home/vestella/miniconda3/envs/my_env2/lib/python3.7/site-packages/xarray/core/indexing.py:1227: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  return self.array[key]


-90.0 89.75
tasmax


/home/vestella/miniconda3/envs/my_env2/lib/python3.7/site-packages/xarray/core/indexing.py:1227: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  return self.array[key]


-90.0 89.75
tasmin


/home/vestella/miniconda3/envs/my_env2/lib/python3.7/site-packages/xarray/core/indexing.py:1227: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  return self.array[key]


-90.0 89.75
sfcWind
/data/AIDL-UPC/seas_cyc_variables_ERA5_v0.3.nc


# compute mean and std of the elevation data

In [6]:
data = open_elevation_data()

In [7]:
land_ocean = xr.where(data==data._FillValue,0,1)
elevation = xr.where(data==data._FillValue,0,data)
elevation_norm = (elevation-elevation.mean())/elevation.std()

In [8]:
land_ocean.to_netcdf('/data/AIDL-UPC/land_ocean_mask_v'+version+'.nc')
print('/data/AIDL-UPC/land_ocean_mask_v'+version+'.nc')

elevation_norm.to_netcdf('/data/AIDL-UPC/elevation_norm_v'+version+'.nc')
print('/data/AIDL-UPC/elevation_norm_v'+version+'.nc')

elevation.to_netcdf('/data/AIDL-UPC/elevation_v'+version+'.nc')
print('/data/AIDL-UPC/elevation_v'+version+'.nc')

/data/AIDL-UPC/land_ocean_mask_v0.3.nc
/data/AIDL-UPC/elevation_norm_v0.3.nc
/data/AIDL-UPC/elevation_v0.3.nc


# mean and std of precipitation

In [20]:
path_precip  = '/data/AIDL-UPC/daymet_v4_daily/prcp/used/short/'
file_precip = 'daymet_v4_daily_na_prcp_'

data_precip = xr.open_mfdataset(path_precip + file_precip + '????_sel200.nc', engine = 'netcdf4' )

vars_name = 'prcp'

In [22]:
data_precip

<xarray.Dataset>
Dimensions:                  (x: 200, y: 200, time: 7665, nv: 2)
Coordinates:
  * x                        (x) float32 9.018e+05 -2.784e+06 ... -1.367e+06
  * y                        (y) float32 1.657e+06 3.623e+06 ... 1.642e+06
    lat                      (time, y, x) float32 57.2 51.14 ... 55.47 56.11
    lon                      (time, y, x) float32 -84.76 -143.2 ... -122.7
  * time                     (time) datetime64[ns] 2000-01-01T12:00:00 ... 20...
Dimensions without coordinates: nv
Data variables:
    yearday                  (time) int16 dask.array<chunksize=(365,), meta=np.ndarray>
    time_bnds                (time, nv) datetime64[ns] dask.array<chunksize=(365, 2), meta=np.ndarray>
    lambert_conformal_conic  (time) int16 -32767 -32767 -32767 ... -32767 -32767
    prcp                     (time, y, x) float32 dask.array<chunksize=(365, 200, 200), meta=np.ndarray>
Attributes:
    start_year:        2000
    source:            Daymet Software Version 4.0
    Version_software:  Daymet Software Version 4.0
    Version_data:      Daymet Data Version 4.0
    Conventions:       CF-1.6
    citation:          Please see http://daymet.ornl.gov/ for current Daymet ...
    references:        Please see http://daymet.ornl.gov/ for current informa...

In [21]:
mean_single = data_precip[vars_name].mean(dim = 'time').values
std_single = data_precip[vars_name].std(dim = 'time').values
mean_year = data_precip[vars_name].groupby('time.dayofyear').mean('time').load()

/home/vestella/miniconda3/envs/my_env2/lib/python3.7/site-packages/dask/array/numpy_compat.py:40: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)


In [31]:
da = xr.Dataset(
data_vars=dict(
    mean=([ "y", "x" ], mean_single),
    std=([  "y", "x"], std_single),
    seas_cyc = ([ "dayofyear",  "y", "x"], mean_year.values),
            ),
coords=dict(
    x = data_precip.x.data,
    y = data_precip.y.data,
    lon = ([ "y", "x"], data_precip.lon.data[0,:,:]),
    lat = ([ "y", "x"], data_precip.lat.data[0,:,:]),
    dayofyear = np.arange(1,366)
),
attrs=dict(
    description="seasonal cycle, std and mean of the period 2000-2018 for precipitation high res",
),
)

da.to_netcdf('/data/AIDL-UPC/std_mean_daymet_v4_daily_v'+version+'.nc')
print('/data/AIDL-UPC/std_mean_daymet_v4_daily_v'+version+'.nc')

/data/AIDL-UPC/std_mean_daymet_v4_daily_v0.3.nc


In [5]:
lat_max = 720
for ivar in range(1):
    data_era5 = xr.open_mfdataset(path + vars_name[ivar]+'_day/'+vars_name[ivar] + '_*', engine = 'zarr' )
    data_era5 = data_era5.sel(time = slice(*['2000', '2018']))
    if lat_max!= data_era5.lat.shape[0]:
        data_era5 = data_era5.isel(lat = np.arange(720))
    mean_year = data_era5[vars_name[ivar]].groupby('time.dayofyear').mean('time').load()

/home/vestella/miniconda3/envs/my_env2/lib/python3.7/site-packages/xarray/core/indexing.py:1227: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  return self.array[key]
